In [1]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join
from PIL import Image
import pandas as pd

In [2]:
import sys
sys.path.append("../")

from src.constants import *
from src.training_utils.dataset import *

In [3]:
BASE_DIR_IMG = f"{PATH}/data/imagenes_diagramas/"
BASE_DIR_XML = f"{PATH}/data/xml_diagramas/"

# CSV = f"{PATH}/data/tiles/data_cardinalidades_sliced.csv"

In [7]:
train_df = pd.read_csv("/home/nacho/TFI-Cazcarra/data/csv/augmented_train_diagramas.csv", header=None)
train_df.columns = ['image_path', 'xmin', 'ymin', 'xmax', 'ymax', 'label']
train_df = train_df[train_df['label']=="tabla"] 
test_df = pd.read_csv("/home/nacho/TFI-Cazcarra/data/csv/augmented_test_diagramas.csv")
test_df = test_df[test_df['label']=="tabla"] 

In [8]:
le_dict = {'tabla': 1}

In [10]:
CLASSES = ['tabla']
DIR = [BASE_DIR_IMG]

In [11]:
def get_images_in_dir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '*.png'):
        image_list.append(filename)
    return image_list

In [12]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [15]:
def convert_annotation(dir_path, output_path, image_path):
    '''
    dir_path -> image_slices/
    output_path -> yolo_tablas/train o yolo_tablas/test
    image_path -> imagen .png
    '''
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0] # El nombre de la imagen en limpio

    in_file = open(BASE_DIR_XML +  basename_no_ext + '.xml')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')

    w,h = Image.open(image_path).size
    print(output_path + basename_no_ext)
    with open(output_path + basename_no_ext + '.txt', 'w+') as out_file:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in CLASSES:
                continue
            cls_id = le_dict[cls] - 1 # Para que empiecen desde 0
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
                 float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w,h), b)
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
    out_file.close()
cwd = getcwd()

In [17]:
for dir_path in DIR:
    full_dir_path = dir_path
    output_path = f"{PATH}/data/yolo_tablas/"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = get_images_in_dir(full_dir_path)       
#     train_file = open(full_dir_path +  "train/" + '.txt', 'w')
#     test_file = open(full_dir_path + "test/" + '.txt', 'w')

    for image_path in image_paths:
        current_folder = "train/" if image_path in train_df['image_path'].unique() else "test/"
        out_path = output_path + current_folder       
        
        convert_annotation(full_dir_path, out_path, image_path)

    print("Finished processing: " + dir_path)

/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-6
/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-27
/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-58
/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-13
/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-14
/home/nacho/TFI-Cazcarra/data/yolo_tablas/test/ERDiagramsMySQL-18
/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-12
/home/nacho/TFI-Cazcarra/data/yolo_tablas/test/ERDiagramsMySQL-8
/home/nacho/TFI-Cazcarra/data/yolo_tablas/test/ERDiagramsMySQL-17
/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-26
/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-48
/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-9
/home/nacho/TFI-Cazcarra/data/yolo_tablas/train/ERDiagramsMySQL-51
/home/nacho/TFI-Cazcarra/data/yolo_tablas/test/ERDiagramsMySQL-22
/home/nacho/TFI-Cazcarra/data/yolo_tablas/test/ERDiagramsMySQL-65
/ho

## Copy files to train/ and test/

In [18]:
import shutil
all_images = get_images_in_dir(dir_path)

In [21]:
for img in all_images:
    current_folder = "train" if img in train_df['image_path'].unique() else "test"
    dst = f"/home/nacho/TFI-Cazcarra/data/yolo_tablas/{current_folder}/images/{os.path.basename(img)}"
    print(img, dst)
    shutil.copyfile(img, dst)

/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-6.png /home/nacho/TFI-Cazcarra/data/yolo_tablas/train/images/ERDiagramsMySQL-6.png
/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-27.png /home/nacho/TFI-Cazcarra/data/yolo_tablas/train/images/ERDiagramsMySQL-27.png
/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-58.png /home/nacho/TFI-Cazcarra/data/yolo_tablas/train/images/ERDiagramsMySQL-58.png
/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-13.png /home/nacho/TFI-Cazcarra/data/yolo_tablas/train/images/ERDiagramsMySQL-13.png
/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-14.png /home/nacho/TFI-Cazcarra/data/yolo_tablas/train/images/ERDiagramsMySQL-14.png
/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-18.png /home/nacho/TFI-Cazcarra/data/yolo_tablas/test/images/ERDiagramsMySQL-18.png
/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-12.png /home/nacho/TFI-Cazcarra/data/y

In [ ]:
## Manually move .txts to labels/